In [40]:
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot
import pandas as pd
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

In [79]:
def boosting(trainfilepath,testfilepath,predfilepath): 
    data_train=pd.read_csv(trainfilepath, index_col=False)
    data_test=pd.read_csv(testfilepath, index_col=False)
    data_pred=pd.read_csv(predfilepath, index_col=False)
    data_pred=data_pred[data_pred.RegionName.apply(lambda x:x[-7:][:4])=='2019']


    data_trainx=data_train.iloc[:,4:]
    data_testx=data_test.iloc[:,4:]
    data_predx=data_pred.iloc[:,4:]
    data_trainy=data_train.iloc[:,3]
    data_testy=data_test.iloc[:,3]
    data_predy=data_pred.iloc[:,3]
    data_predy=np.log(data_predy)
    data_testy=np.log(data_testy)
    data_trainy=np.log(data_trainy)
    model = GradientBoostingRegressor(random_state=2)

    params = dict()
    params['n_estimators'] = [10, 50, 100, 500,1000]
    params['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
    params['subsample'] = [0.5, 0.7, 1.0]
    params['max_depth'] = [3, 7, 9]
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='r2')
    model.fit(data_trainx, data_trainy)
    grid_result = grid_search.fit(data_trainx, data_trainy)

    print("Best Training CV Score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print("\nBest Testing Score: %f " %grid_search.score(data_testx,data_testy))
    print("\nBest Predicting Score: %f " % grid_search.score(data_predx,data_predy))

    model = GradientBoostingRegressor(**grid_result.best_params_,random_state=2)
    model.fit(data_trainx, data_trainy)
    print(pd.DataFrame({'Feature':data_trainx.columns, 'Feature_Importance':model.feature_importances_}).sort_values(by=['Feature_Importance'],ascending=False))



    return grid_search


In [80]:
boosting('TESTDATAnewcols7.17_3year.csv','TRAINDATAnewcols7.17_3year.csv','PREDICTDATAnewcols7.17_3year.csv')

Best Training CV Score: 0.549829 using {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1000, 'subsample': 0.5}

Best Testing Score: 0.670018 

Best Predicting Score: 0.600936 
                                    Feature  Feature_Importance
5                              mobile_homes            0.155059
3              million_dollar_housing_units            0.084407
4                dwellings_50_or_more_units            0.073060
44        Perc_commuter_by_public_transport            0.072886
34               Perc_commuters_over_45_min            0.049094
42                               Perc_asian            0.039749
29                        dwellings_1_units            0.029814
27                           Percent_Female            0.029219
45                      Perc_worked_at_home            0.025153
51               HIFL_transit_station_count            0.023158
39                 Perc_vacat_housing_units            0.022481
2                             median_income     

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=GradientBoostingRegressor(random_state=2), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_depth': [3, 7, 9],
                         'n_estimators': [10, 50, 100, 500, 1000],
                         'subsample': [0.5, 0.7, 1.0]},
             scoring='r2')

In [81]:
boosting('TESTDATAnewcols7.18_3year.csv','TRAINDATAnewcols7.18_3year.csv','PREDICTDATAnewcols7.18_3year.csv')



KeyboardInterrupt: 